In [1]:
import requests
import math
import pandas as pd
import json
import os

In [3]:
os.system('mkdir -p cache')

1

In [2]:
import re
from tokenizers import normalizers
from tokenizers.normalizers import BertNormalizer, Sequence, Strip
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Whitespace

normalizer = Sequence([BertNormalizer(clean_text=True,
        handle_chinese_chars=True,
        strip_accents=True,
        lowercase=True), Strip()])
pre_tokenizer = pre_tokenizers.Sequence([Whitespace()])

def normalize(x, min_length = 1):
    if not isinstance(x, str):
        return ''
    normalized = normalizer.normalize_str(x)
    normalized = normalized.replace('\n', ' ')
    normalized = re.sub(' +', ' ', normalized)
    return " ".join([e[0] for e in pre_tokenizer.pre_tokenize_str(normalized) if len(e[0]) > min_length])

### Nettoyer lib entreprises pour query

In [5]:
import urllib.parse

In [16]:
companies = pd.read_csv('liste_entreprises.csv',sep=';', index_col=0)
companies = companies.reset_index()  # make sure indexes pair with number of rows
companies.columns

Index(['index', 'name', 'source', 'type'], dtype='object')

In [43]:
list_queries = []
for company in companies['name']:
    company = company.replace('.','')
    company = company.replace(')',' ')
    company = company.replace('(',' ')
    a = normalize(company)
    list_queries.append(a)
companies['query'] = list_queries
companies.to_csv('data/liste_entreprises_queries.csv',sep=';')
companies.sample(10)

,index,name,source,type,query
15030,10506,Civil Aviation Directorateof the Republic of S...,wikidata,public company,civil aviation directorateof the republic of s...
10357,5833,Senomyx,wikidata,public company,senomyx
5890,1366,Orix,wikidata,public company,orix
15873,11349,Binakarya Jaya Abadi,wikidata,public company,binakarya jaya abadi
17841,13317,Q115310661,wikidata,public company,q115310661
4125,4125,SOCIETE POUR L'INFORMATIQUE INDUSTRIELLE,scanR,CII;CIR,societe pour informatique industrielle
4798,274,HSBC,wikidata,public company,hsbc
14301,9777,Q20062810,wikidata,public company,q20062810
7580,3056,SinnerSchrader,wikidata,public company,sinnerschrader
2335,2335,ADACSYS,scanR,i-LAB,adacsys


In [44]:
list_queries = list(companies['query'])
len(list_queries)

17918

In [46]:
index = list_queries.index('unitronics 1989 rg ltd')
index
del list_queries[0:index]
list_queries

['unitronics 1989 rg ltd',
 'lithia motors',
 'rs public company limited',
 'littelfuse',
 'liveperson inc',
 'big shopping centers ltd',
 'livechat software',
 'livermore investment group',
 'general cable',
 'lojack',
 'isras investment company ltd',
 'nevsun resources',
 'newalliance bank',
 'new horizons computer learning centers',
 'new jersey resources',
 'new mountain capital',
 'new relic',
 'vision values holdings limited',
 'new york company',
 'stingray radio',
 'newfield exploration',
 'newmark group',
 'neto malinda trading ltd',
 'nexstar media group',
 'next digital',
 'nextdoor',
 'nib health funds',
 'nichicon',
 'fattal holdings 1998 ltd',
 'nichols plc',
 'niko resources',
 'nippon chemi con',
 'nh foods',
 'nissan chemical corporation',
 'nisshinbo holdings',
 'noble corporation',
 'noodles company',
 'norbord',
 'nord anglia education',
 'northwestern corporation',
 'north american palladium',
 'north drilling',
 'israel ports development assets company',
 'northea

In [47]:
len(list_queries)

8204

In [6]:
first_year = 2000

In [49]:
# assert (affiliation1_type in ['raw_affiliation_string', 'institutions.country_code', 'institutions.ror'])

def get_filename(affiliation1_str, affiliation1_type, affiliation2_str, affiliation2_type, thematic, random_size ):
    filename = f'cache/{normalize(affiliation1_str)}_{normalize(affiliation2_str)}_sample{random_size}_{normalize(thematic)}.json'.replace(' ', '_')
    return filename

In [50]:
BASE_URL = f"https://api.openalex.org/works?filter=publication_year:{first_year}-"

In [51]:
def get_search_field(f):
    if f in ['raw_affiliation_string']:
        return 'raw_affiliation_string.search'
    return f

def get_data(affiliation1_str, affiliation1_type, affiliation2_str, affiliation2_type, thematic ):
    url = BASE_URL + f",{get_search_field(affiliation1_type)}:{affiliation1_str}"
    if affiliation2_str:
        url += f",{get_search_field(affiliation2_type)}:{affiliation2_str}"
    if thematic:
        random_size = None
        url += f"&search={thematic}"
    else:
        random_size = 1000
        url += f"&sample={random_size}&seed=0"
        
    print(url)
    filename = get_filename(affiliation1_str, affiliation1_type, affiliation2_str, affiliation2_type, 
                            thematic, random_size )
    print(filename)
    try:
        data = json.load(open(filename, 'r'))
    except:
        results = []
        res = requests.get(url).json()
        nb_results = res['meta']['count']
        print(nb_results)
        nb_page = math.ceil(nb_results / res['meta']['per_page'])
        results = res['results']
        for p in range(2, nb_page + 1):
            print(p, end=',')
            res = requests.get(url+f"&page={p}").json()
            results += res['results']
        assert(len(results) == nb_results)
        data = {'results': results}
        json.dump(data, open(filename, 'w'))
    return data['results']

In [52]:
all_works = pd.DataFrame()
thematic = None
for company in list_queries:
    data = get_data(company, 'raw_affiliation_string', 'france', 'raw_affiliation_string', thematic)
    df = pd.DataFrame(data)
    df['query'] = company
    all_works = pd.concat([all_works, df])

https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:unitronics 1989 rg ltd,raw_affiliation_string.search:france&sample=1000&seed=0
cache/unitronics_1989_rg_ltd_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:lithia motors,raw_affiliation_string.search:france&sample=1000&seed=0
cache/lithia_motors_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:rs public company limited,raw_affiliation_string.search:france&sample=1000&seed=0
cache/rs_public_company_limited_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:littelfuse,raw_affiliation_string.search:france&sample=1000&seed=0
cache/littelfuse_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:liveperson inc,raw_affiliation_string.search:

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:nord anglia education,raw_affiliation_string.search:france&sample=1000&seed=0
cache/nord_anglia_education_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:northwestern corporation,raw_affiliation_string.search:france&sample=1000&seed=0
cache/northwestern_corporation_france_sample1000_.json
63
2,3,https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:north american palladium,raw_affiliation_string.search:france&sample=1000&seed=0
cache/north_american_palladium_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:north drilling,raw_affiliation_string.search:france&sample=1000&seed=0
cache/north_drilling_france_sample1000_.json
2
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:israel ports

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:olympic steel,raw_affiliation_string.search:france&sample=1000&seed=0
cache/olympic_steel_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:omega protein,raw_affiliation_string.search:france&sample=1000&seed=0
cache/omega_protein_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:omnicare,raw_affiliation_string.search:france&sample=1000&seed=0
cache/omnicare_france_sample1000_.json
2
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:onebeacon,raw_affiliation_string.search:france&sample=1000&seed=0
cache/onebeacon_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:one horizon group,raw_affiliation_string.search:france&sample=1000&seed=0
cache/one_horizon_group

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:thai oil,raw_affiliation_string.search:france&sample=1000&seed=0
cache/thai_oil_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:clal insurance enterprises holdings ltd,raw_affiliation_string.search:france&sample=1000&seed=0
cache/clal_insurance_enterprises_holdings_ltd_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:lotte chilsung,raw_affiliation_string.search:france&sample=1000&seed=0
cache/lotte_chilsung_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:melisron ltd,raw_affiliation_string.search:france&sample=1000&seed=0
cache/melisron_ltd_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:lovesac,raw_affiliation_string.search

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:major drilling group international,raw_affiliation_string.search:france&sample=1000&seed=0
cache/major_drilling_group_international_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:mallinckrodt pharmaceuticals,raw_affiliation_string.search:france&sample=1000&seed=0
cache/mallinckrodt_pharmaceuticals_france_sample1000_.json
7
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:mantech international,raw_affiliation_string.search:france&sample=1000&seed=0
cache/mantech_international_france_sample1000_.json
2
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:mannkind corporation,raw_affiliation_string.search:france&sample=1000&seed=0
cache/mannkind_corporation_france_sample1000_.json
2
https://api.openalex.org/works?filter=publication_year:2000-,raw_aff

...

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310892,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310892_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310894,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310894_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310896,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310896_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310901,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310901_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310902,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310902_france_sample1000_.

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310965,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310965_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310967,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310967_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310971,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310971_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310975,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310975_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310977,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310977_france_sample1000_.

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310623,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310623_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310624,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310624_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310627,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310627_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310631,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310631_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310633,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310633_france_sample1000_.

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310731,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310731_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310738,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310738_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310740,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310740_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310742,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310742_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310743,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310743_france_sample1000_.

0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310830,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310830_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310836,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310836_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310837,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310837_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310838,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310838_france_sample1000_.json
0
https://api.openalex.org/works?filter=publication_year:2000-,raw_affiliation_string.search:q115310839,raw_affiliation_string.search:france&sample=1000&seed=0
cache/q115310839_france_sample1000_.

In [53]:
all_works

,query,id,doi,title,display_name,relevance_score,publication_year,publication_date,ids,primary_location,...,alternate_host_venues,referenced_works,related_works,ngrams_url,abstract_inverted_index,cited_by_api_url,counts_by_year,updated_date,created_date,is_authors_truncated
0,general cable,https://openalex.org/W2893008174,https://doi.org/10.1109/icd.2018.8468375,Thermoplastic blend balanced property developm...,Thermoplastic blend balanced property developm...,0.355244,2018.0,2018-07-01,{'openalex': 'https://openalex.org/W2893008174...,"{'is_oa': None, 'landing_page_url': 'https://d...",...,"[{'id': 'https://openalex.org/S4306422626', 'd...",[https://openalex.org/W1970619531],"[https://openalex.org/W1987888414, https://ope...",https://api.openalex.org/works/W2893008174/ngrams,"{'The': [0], 'improvement': [1], 'of': [2, 7, ...",https://api.openalex.org/works?filter=cites:W2...,[],2023-02-26T19:33:28.239278,2018-10-05,NaN
1,general cable,https://openalex.org/W3187807764,None,Second wind in the offshore wind industry,Second wind in the offshore wind industry,0.013792,2011.0,2011-06-01,{'openalex': 'https://openalex.org/W3187807764...,"{'is_oa': None, 'landing_page_url': 'https://i...",...,"[{'id': None, 'display_name': None, 'type': No...",[],"[https://openalex.org/W1559412770, https://ope...",https://api.openalex.org/works/W3187807764/ngrams,None,https://api.openalex.org/works?filter=cites:W3...,[],2023-02-19T02:08:03.644530,2021-08-16,NaN
0,new jersey resources,https://openalex.org/W2508349122,https://doi.org/10.1038/nclimate3298,Priority for the worse-off and the social cost...,Priority for the worse-off and the social cost...,0.995913,2017.0,2017-06-01,{'openalex': 'https://openalex.org/W2508349122...,"{'is_oa': None, 'landing_page_url': 'https://d...",...,"[{'id': 'https://openalex.org/S183584863', 'di...","[https://openalex.org/W1518602771, https://ope...","[https://openalex.org/W108925530, https://open...",https://api.openalex.org/works/W2508349122/ngrams,"{'The': [0, 13, 34, 50], 'social': [1, 20, 48]...",https://api.openalex.org/works?filter=cites:W2...,"[{'year': 2022, 'cited_by_count': 12}, {'year'...",2023-02-27T01:00:59.216293,2016-09-16,NaN
1,new jersey resources,https://openalex.org/W1974472004,https://doi.org/10.1371/journal.pntd.0002615,"Animal Reservoir, Natural and Socioeconomic Va...","Animal Reservoir, Natural and Socioeconomic Va...",0.968341,2014.0,2014-01-09,{'openalex': 'https://openalex.org/W1974472004...,"{'is_oa': True, 'landing_page_url': 'https://d...",...,"[{'id': 'https://openalex.org/S46544255', 'dis...","[https://openalex.org/W1938339774, https://ope...","[https://openalex.org/W1580426574, https://ope...",https://api.openalex.org/works/W1974472004/ngrams,"{'China': [0], 'has': [1], 'the': [2, 19, 42, ...",https://api.openalex.org/works?filter=cites:W1...,"[{'year': 2023, 'cited_by_count': 1}, {'year':...",2023-03-01T10:25:36.572529,2016-06-24,NaN
2,new jersey resources,https://openalex.org/W4297810866,https://doi.org/10.1175/bams-d-22-0092.1,Global Climate,Global Climate,0.908126,2022.0,2022-08-01,{'openalex': 'https://openalex.org/W4297810866...,"{'is_oa': True, 'landing_page_url': 'https://d...",...,"[{'id': 'https://openalex.org/S140302479', 'di...",[],"[https://openalex.org/W1564116264, https://ope...",https://api.openalex.org/works/W4297810866/ngrams,None,https://api.openalex.org/works?filter=cites:W4...,"[{'year': 2022, 'cited_by_count': 1}]",2023-02-27T16:14:19.996784,2022-10-01,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,machi machi,https://openalex.org/W2790987055,https://doi.org/10.1007/s00774-018-0910-7,Does Trabecular Bone Score (TBS) improve the p...,Does Trabecular Bone Score (TBS) improve the p...,0.000372,2019.0,2019-01-01,{'openalex': 'https://openalex.org/W2790987055...,"{'is_oa': None, 'landing_page_url': 'https://d...",...,"[{'id': 'https://openalex.org/S178027841', 'di...","[https://openalex.org/W1858257286, https://ope..."

In [54]:
size = len(all_works)
all_works.to_csv(f'data/works_{size}.csv', sep=';')